# Problème : Organisation d’un championnat

**Auteurs**
- 28627904 LUONG Ethan
- 21304385 PHAM Louis-Antoine

**Objectif.** Le but de ce TME est de programmer la grille de match d’un championnat. On considère qu’il y a $n_e$ équipes participantes. Les matchs de championnat peuvent se d´erouler le mercredi ou le dimanche. Le championnat dure (au maximum) ns semaines (avec 2 matchs par semaine), soit $n_j = 2\times n_s$ jours de match.

Les matchs se font sur le terrain de l’une des deux équipes, qui est considérée comme jouant *à domicile*. L’autre équipe joue *à l’extérieur*. Etant donné les déplacements et la fatigue du match, chaque équipe ne peut jouer plus d’un match par jour. Sur la durée du championnat, chaque équipe doit rencontrer l’ensemble des autres équipes une fois *à domicile* et une fois *à l’extérieur*, soit exactement 2 matchs par équipe adverse.

Le but est de produire un planning des matchs, indiquant pour chaque jour quelles équipes s’affrontent en précisant où ont lieu les matchs.

Afin d’avoir une solution générique, paramétrable par $n_j$ et $n_e$, et en raison du nombre important de variables propositionnelles et de contraintes à considérer, les exercices suivants ont pour objectif d’écrire des fonctions permettant de générer le fichier DIMACS représentant le problème. Le langage de programmation est au choix.

## Modélisation

Chaque équipe correspond à un numéro entre $0$ et $n_e−1$. De même, chaque jour de match correspond à un numéro entre $0$ et $n_j−1$. On considère que le championnat commence un mercredi. Les jours pairs ($0$ compris), correspondent donc à des mercredis, tandis que les jours impairs correspondent à des dimanches.

On représente par des variables propositionnelles $m{j,x,y}$ le fait qu’il y ait (ou non) un match entre l’´equipe $x$, jouant *à domicile*, et l’équipe $y$ au jour $j$.

Au format DIMACS toutes les variables doivent être numérotées. On propose de coder la variable $m{j,x,y}$ par $v_k$ où $k = j\times n_e^2 + x\times n_e + y + 1$.

In [35]:
# Il y a mjxy = nj*ne*ne sans la contrainte x != y

def codage(ne, nj, j, x, y):
    assert(ne > 0)
    assert(j <= nj-1 and j >= 0)
    assert(x <= ne-1 and x >= 0)
    assert(x <= ne-1 and x >= 0)
    return j*ne**2 + x*ne + y + 1

k = codage(3, 2, 1, 2, 1)
print('Codage : ', k)

# On peut voir k comme un mot en base ne avec 3 chiffres : j, x, y
def decodage(k, ne):
    assert(ne > 0)
    y = (k-1) % ne
    x = ((k-1) // ne) % ne
    j = ((k-1) // ne**2) % ne
    return j, x, y

print('Decodage : ', decodage(k, 3))

Codage :  17
Decodage :  (1, 2, 1)


In [36]:
print('Tests des intervalles de valeurs :')
try:
    codage(0, 2, 2, 2, 1)
except AssertionError:
    print('\tIl faut ne > 0')

try:
    codage(1, 2, 2, 2, 1)
except AssertionError:
    print('\tx et y doivent être strictement plus petits que ne')

try:
    codage(3, 2, 4, 2, 1)
except AssertionError:
    print('\tj doit être strictement plus petit que nj')

Tests des intervalles de valeurs :
	Il faut ne > 0
	x et y doivent être strictement plus petits que ne
	j doit être strictement plus petit que nj


In [37]:
print('Tests codage / décodage')
assert(decodage(codage(3, 3, 2, 0, 0), 3) == (2, 0, 0))
assert(decodage(codage(3, 3, 1, 2, 1), 3) == (1, 2, 1))
print('Tests réussis !')

Tests codage / décodage
Tests réussis !


## Génération d'un planning de matchs

Le but de cet exercice est de résoudre le problème par une méthode SAT directe. Il s’agit donc de traduire et encoder le problème en SAT, d’utiliser glucose pour r´esoudre le problème ainsi traduit, puis de décoder le modèle fourni en un format adapté au problème et lisible.

In [38]:
def au_moins(L):
    if len(L) == 0:
        return ''
    return ' '.join([str(v) for v in L]) + ' 0\n'

# Encodage par paires
def au_plus(L):
    contrainte = ''
    for i in range(len(L)):
        for j in range(i+1, len(L)):
            contrainte += f'{-L[i]} {-L[j]} 0\n'
    return contrainte

In [39]:
print('Tests des valeurs de retour de au_moins')
assert(au_moins([]) == '')
assert(au_moins([1, 2, 3]) == '1 2 3 0\n')
print('Tests réussis !')

Tests des valeurs de retour de au_moins
Tests réussis !


In [40]:
print('Tests des valeurs de retour de au_plus')
assert(au_plus([]) == '')
assert(au_plus([1, 2, 3]) == '-1 -2 0\n-1 -3 0\n-2 -3 0\n')
print('Tests réussis !')

Tests des valeurs de retour de au_plus
Tests réussis !


In [41]:
def encoderC1(ne, nj):
    assert(ne > 0 and nj > 0)
    C1 = ''
    for j in range(nj):
        for x in range(ne):
            mjxy = [codage(ne, nj, j, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j, y, x) for y in range(ne) if y != x]
            C1 += au_plus(mjxy)
    return C1

C1 = encoderC1(3, 4)
print('Nombres de contraintes C1 : ', len(C1.split('\n'))-1)
print('Contraintes C1 : ', C1)

Nombres de contraintes C1 :  72
Contraintes C1 :  -2 -3 0
-2 -4 0
-2 -7 0
-3 -4 0
-3 -7 0
-4 -7 0
-4 -6 0
-4 -2 0
-4 -8 0
-6 -2 0
-6 -8 0
-2 -8 0
-7 -8 0
-7 -3 0
-7 -6 0
-8 -3 0
-8 -6 0
-3 -6 0
-11 -12 0
-11 -13 0
-11 -16 0
-12 -13 0
-12 -16 0
-13 -16 0
-13 -15 0
-13 -11 0
-13 -17 0
-15 -11 0
-15 -17 0
-11 -17 0
-16 -17 0
-16 -12 0
-16 -15 0
-17 -12 0
-17 -15 0
-12 -15 0
-20 -21 0
-20 -22 0
-20 -25 0
-21 -22 0
-21 -25 0
-22 -25 0
-22 -24 0
-22 -20 0
-22 -26 0
-24 -20 0
-24 -26 0
-20 -26 0
-25 -26 0
-25 -21 0
-25 -24 0
-26 -21 0
-26 -24 0
-21 -24 0
-29 -30 0
-29 -31 0
-29 -34 0
-30 -31 0
-30 -34 0
-31 -34 0
-31 -33 0
-31 -29 0
-31 -35 0
-33 -29 0
-33 -35 0
-29 -35 0
-34 -35 0
-34 -30 0
-34 -33 0
-35 -30 0
-35 -33 0
-30 -33 0



In [42]:
def encoderC2(ne, nj):
    assert(ne > 0 and nj > 0)
    C2 = ''
    for x in range(ne):
        for y in range(x+1, ne):
            mjxy = [codage(ne, nj, j, x, y) for j in range(nj)]
            mjyx = [codage(ne, nj, j, y, x) for j in range(nj)]
            C2 += au_moins(mjxy) + au_plus(mjxy)
            C2 += au_moins(mjyx) + au_plus(mjyx)
    return C2

C2 = encoderC2(3, 4)
print('Nombres de contraintes C2 : ', len(C2.split('\n'))-1)
print('Contraintes C2 : ', C2)

Nombres de contraintes C2 :  42
Contraintes C2 :  2 11 20 29 0
-2 -11 0
-2 -20 0
-2 -29 0
-11 -20 0
-11 -29 0
-20 -29 0
4 13 22 31 0
-4 -13 0
-4 -22 0
-4 -31 0
-13 -22 0
-13 -31 0
-22 -31 0
3 12 21 30 0
-3 -12 0
-3 -21 0
-3 -30 0
-12 -21 0
-12 -30 0
-21 -30 0
7 16 25 34 0
-7 -16 0
-7 -25 0
-7 -34 0
-16 -25 0
-16 -34 0
-25 -34 0
6 15 24 33 0
-6 -15 0
-6 -24 0
-6 -33 0
-15 -24 0
-15 -33 0
-24 -33 0
8 17 26 35 0
-8 -17 0
-8 -26 0
-8 -35 0
-17 -26 0
-17 -35 0
-26 -35 0



In [43]:
import os

def encoder(ne, nj):
    assert(ne > 0 and nj > 0)
    return f'p cnf {nj*ne*ne}\n{encoderC1(ne, nj)}{encoderC2(ne, nj)}'

def programme(ne, nj):
    os.system(f'{encoder(ne, nj)} > glucose > res.txt')

def dico_equipe(filename='equipes.txt'):
    dico = {}
    with open(filename, 'r') as f:
        lignes = f.readlines()
        for i, ligne in enumerate(lignes):
            dico[i] = ligne
        return dico

def decoder(filename_results='res.txt', filename_equipes='equipes.txt'):
    equipes = dico_equipe(filename_equipes)
    with open(filename_results, 'r') as f:
        lignes = f.readlines()
        if lignes[0] == 'SAT\n':
            print('Solution :')
            for ligne in lignes[1:]:
                if ligne != '0\n':
                    j, x, y = decodage(int(ligne), 3)
                    print(f'Jour {j} : {equipes[x]} (dom.) vs {equipes[y]} (ext.)')
            return 'SAT'
        else:
            return 'UNSAT'

## Optimisation du nombre de jours

In [44]:
import signal

class TimeoutError(Exception):
    pass

def timeout(ne, timeout_duration=10.0):
    signal.signal(signal.SIGNALRM)
    signal.alarm(timeout_duration)
    try:
        for nj in range(ne*ne):
            signal.alarm(0)
            if programme(ne, nj) == 'SAT':
                break
    except TimeoutError:
        print('Timeout')
    return nj

def optimiser_nj_selon_ne():
    for ne in range(3, 11):
        print(f'ne = {ne}, nj = {timeout(ne)}')

## Équilibrer les déplacements et les week-ends

In [45]:
from itertools import combinations

def au_plus_etendu(L, k):
    contrainte = ''
    combinaisons = list(combinations(L, k+1))
    for sous_liste in combinaisons:
        contrainte += ' '.join([str(-v) for v in sous_liste]) + ' 0\n'
    return contrainte

def au_moins_etendu(L, k):
    N = len(L)
    return au_plus_etendu([-v for v in L], N-k)

print(au_plus_etendu([1, 2, 3, 4], 2))
print(au_moins_etendu([1, 2, 3, 4], 2))

-1 -2 -3 0
-1 -2 -4 0
-1 -3 -4 0
-2 -3 -4 0

1 2 3 0
1 2 4 0
1 3 4 0
2 3 4 0



In [46]:
import math

def encoderC3(ne, nj, p_ext=0.5, p_dom=0.4):
    assert(ne > 0 and nj > 0)
    assert(p_ext >= 0 and p_ext <= 1)
    assert(p_dom >= 0 and p_dom <= 1)
    C3 = ''
    for x in range(ne):
        L_ext = [codage(ne, nj, dimanche, y, x) for y in range(ne) if y != x for dimanche in range(1, nj, 2)]
        C3 += au_moins_etendu(L_ext, math.ceil(p_ext*(ne-1)))
        L_dom = [codage(ne, nj, dimanche, x, y) for y in range(ne) if y != x for dimanche in range(1, nj, 2)]
        C3 += au_moins_etendu(L_dom, math.ceil(p_dom*(ne-1)))
    return C3

C3 = encoderC3(3, 4)
print('Nombres de contraintes C3 : ', len(C3.split('\n'))-1)
print('Contraintes C3 : ', C3)

Nombres de contraintes C3 :  6
Contraintes C3 :  13 31 16 34 0
11 29 12 30 0
11 29 17 35 0
13 31 15 33 0
12 30 15 33 0
16 34 17 35 0



In [47]:
def encoderC4(ne, nj):
    assert(ne > 0 and nj > 0)
    C4 = ''
    for x in range(ne):
        for j in range(nj-2):
            L_ext = [codage(ne, nj, j, y, x) for y in range(ne) if y != x] + [codage(ne, nj, j+1, y, x) for y in range(ne) if y != x] + [codage(ne, nj, j+2, y, x) for y in range(ne) if y != x]
            C4 += au_plus_etendu(L_ext, 2)
            L_dom = [codage(ne, nj, j, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j+1, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j+2, x, y) for y in range(ne) if y != x]
            C4 += au_plus_etendu(L_dom, 2)
    return C4

C4 = encoderC4(3, 4)
print('Nombres de contraintes C4 : ', len(C4.split('\n'))-1)
print('Contraintes C4 : ', C4)

Nombres de contraintes C4 :  240
Contraintes C4 :  -4 -7 -13 0
-4 -7 -16 0
-4 -7 -22 0
-4 -7 -25 0
-4 -13 -16 0
-4 -13 -22 0
-4 -13 -25 0
-4 -16 -22 0
-4 -16 -25 0
-4 -22 -25 0
-7 -13 -16 0
-7 -13 -22 0
-7 -13 -25 0
-7 -16 -22 0
-7 -16 -25 0
-7 -22 -25 0
-13 -16 -22 0
-13 -16 -25 0
-13 -22 -25 0
-16 -22 -25 0
-2 -3 -11 0
-2 -3 -12 0
-2 -3 -20 0
-2 -3 -21 0
-2 -11 -12 0
-2 -11 -20 0
-2 -11 -21 0
-2 -12 -20 0
-2 -12 -21 0
-2 -20 -21 0
-3 -11 -12 0
-3 -11 -20 0
-3 -11 -21 0
-3 -12 -20 0
-3 -12 -21 0
-3 -20 -21 0
-11 -12 -20 0
-11 -12 -21 0
-11 -20 -21 0
-12 -20 -21 0
-13 -16 -22 0
-13 -16 -25 0
-13 -16 -31 0
-13 -16 -34 0
-13 -22 -25 0
-13 -22 -31 0
-13 -22 -34 0
-13 -25 -31 0
-13 -25 -34 0
-13 -31 -34 0
-16 -22 -25 0
-16 -22 -31 0
-16 -22 -34 0
-16 -25 -31 0
-16 -25 -34 0
-16 -31 -34 0
-22 -25 -31 0
-22 -25 -34 0
-22 -31 -34 0
-25 -31 -34 0
-11 -12 -20 0
-11 -12 -21 0
-11 -12 -29 0
-11 -12 -30 0
-11 -20 -21 0
-11 -20 -29 0
-11 -20 -30 0
-11 -21 -29 0
-11 -21 -30 0
-11 -29 -30 0
-12 -20 -

In [48]:
def encoder_etendu(ne, nj):
    assert(ne > 0 and nj > 0)
    return f'p cnf {nj*ne*ne}\n{encoderC1(ne, nj)}{encoderC2(ne, nj)}{encoderC3(ne, nj)}{encoderC4(ne, nj)}'

def programme_etendu(ne, nj):
    os.system(f'{encoder_etendu(ne, nj)} > glucose > res.txt')